## TOOLS CON LANGCHAIN E OLLAMA - 1

In questo notebook viene testata la funzionalità di langchain che consente di connettere un LLM con tools esterni al fine di eseguire chiamate a funzioni al fine di estendere le capacità dell'LLM stesso.

E' necessario utilizzare modelli che supportano function callling. Nel nostro caso utilizziamo llama3.2:3b

Vedi 
https://medium.com/@netanel.margalit/basic-llama-3-2-3b-tool-calling-with-langchain-and-ollama-47ad29b11f34


In [ ]:
#pip install langchain langchain_community langchain-ollama ollama

In [3]:
# Consente di interagire con i modelli registrati su Ollama
from langchain_ollama import ChatOllama

# Consente di "decorare" una funzione al fine di renderla una funzione utilizzabile dal modello
from langchain_core.tools import tool

# Rapprensentano i messaggi forniti al modello da l'interlocutore "umano"
from langchain_core.messages import HumanMessage

# Rappresentano i messaggi forniti all'LLM per definirne il "comportamento"
from langchain_core.messages.system import SystemMessage

Definisco 2 funzioni "stupide" al solo fine di vedere come funziona il meccanismo.<br>
E' necessario che le funzioni siano decorate con @tool e che descrivano tra """""" la funzionalità eseguita<br>
La funzione riceve un valore (parsato da humanmessage) esegue il codice e ritorna il risultato al modello

In [4]:
@tool
def pig_latin(phrase) -> str:
    """Returns the pig latin pronounciation of a phrase"""
    def convert_word(word) -> str:
        if word[0] in "aeiouAEIOU":
            return word + "way"
        else:
            for index, letter in enumerate(word):
                if letter in "aeiouAEIOU":
                    return word[index:] + word[:index] + "ay"
            return word + "ay"

    words = phrase.split()
    pig_latin_words = [convert_word(word) for word in words]
    return " ".join(pig_latin_words)


@tool
def reverse(phrase) -> str:
    """Return a phrase in reversed form"""
    return "".join(reversed(phrase))

I tool richiamabili devono essere inseriti in una lista e deve essere fatto il bind di questa lista al modello

In [5]:
tools = [pig_latin, reverse]

model = ChatOllama(model="llama3.2").bind_tools(tools)

Creo una serie di messaggi per definire:
- il comportamento del modello (SystemMessage)
- cosa si richiede al modello (HumanMessage)
    
Il risultato della chiamata conterrà l'input per le chiamate alle funzioni 

In [10]:
messages = [
    SystemMessage("You are provided questions by the human, and the computation of the answers by tools. Give them an answer using the tool results only. Use the personality of Socrates"),
    HumanMessage("How do I to write 'cacca' in pig latin?"),
    HumanMessage("How do I to write 'donkey' in reverse?")
    ]

ai_msg = model.invoke(messages)

print(ai_msg.tool_calls)

[{'name': 'pig_latin', 'args': {'phrase': 'cacca'}, 'id': '4c1c210c-2ab5-4c92-a258-53f66f80a55e', 'type': 'tool_call'}, {'name': 'reverse', 'args': {'phrase': 'donkey'}, 'id': '8824f02d-970d-4ffc-833d-d6c0bd39d114', 'type': 'tool_call'}]


A questo punto è possibile iterare tra le tool_calls ricevute per richiamare le funzioni e vedere le risposte ricevute dalle funzioni

In [11]:
for tool_call in ai_msg.tool_calls:
    
    # Seleziono il tool da utilizzare in base al nome del tool nella risposta
    selected_tool = {"pig_latin": pig_latin, "reverse": reverse}[tool_call["name"].lower()]
    # Invoco il tool selezionato passando come parametro la tool_call
    tool_msg = selected_tool.invoke(tool_call)
    # Aggiungo la risposta della funzione ai messaggi da passare al modello
    messages.append(tool_msg)
    print(tool_msg)

content='accacay' name='pig_latin' tool_call_id='4c1c210c-2ab5-4c92-a258-53f66f80a55e'
content='yeknod' name='reverse' tool_call_id='8824f02d-970d-4ffc-833d-d6c0bd39d114'


Invoco il modello per ricevere la risposta finale

In [13]:
print(model.invoke(messages).content)

Dear friend, it seems that the art of Pig Latin and word reversal is not so complex after all. To translate "cacca" into Pig Latin, we simply move the first consonant (or consonant cluster) to the end and add "ay". A straightforward process indeed!

And, I must say, reversing the letters in "donkey" yields a result that is quite... unique. It seems that the word "donkey" spelled backwards is "yeknod". Ah, the wonders of linguistic manipulation!


Nel notebook successivo utilizzerò un metodo più "robusto" per una maggior sicurezza nella scelta del tool da utilizzare